In [25]:
#pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
import math 

In [27]:
#Load data
dataset = load_dataset("copenlu/answerable_tydiqa")
dataset

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
})

In [28]:
#split data 
train_set = dataset["train"]
validation_set = dataset["validation"]

In [29]:
#preprocess data
train_set = train_set.to_pandas()
validation_set = validation_set.to_pandas()

languages_chosen = ["english"]

train_set = train_set[train_set['language'].isin(languages_chosen)]
validation_set = validation_set[validation_set['language'].isin(languages_chosen)]

In [30]:
def get_answerability(annotations):
    answerable = []
    for annot in annotations:
        if -1 in annot['answer_start']:
            answerable.append(0)
        else:
            answerable.append(1)
    return answerable

train_annotations = train_set['annotations'].tolist()
validation_annotations = validation_set['annotations'].tolist()



train_set['answerable'] = get_answerability(train_annotations)
validation_set['answerable'] = get_answerability(validation_annotations)
train_set.columns

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Index(['question_text', 'document_title', 'language', 'annotations',
       'document_plaintext', 'document_url', 'answerable'],
      dtype='object')

In [31]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model_checkpoint = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/c3772e6d13ecdaf8d1105055f7c89becd6e37590/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labe

In [32]:
def question_parag_combine(questions, paragraphs):
    """
    This function combines the questions and paragraphs into a single text
    Args:
        questions: list of questions
        paragraphs: list of paragraphs
    Returns:
        list of combined questions and paragraphs
    """
    training_data = []
    for index in range(len(questions)):
        training_data += [questions[index] + "\n" + paragraphs[index]]
        
    return training_data

In [33]:
from datasets import DatasetDict, Dataset

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)


d_q = train_set['question_text'].tolist()
d_p = train_set['document_plaintext'].tolist()
training_data = question_parag_combine(d_q,d_p)
training_labels = train_set['answerable'].tolist()

d_q = validation_set['question_text'].tolist()
d_p = validation_set['document_plaintext'].tolist()
validation_data = question_parag_combine(d_q,d_p)
validation_labels = validation_set['answerable'].tolist()



data_set = {}
sets = [['train',train_set['question_text'].tolist(), training_labels], ['val', validation_set['question_text'].tolist(), validation_labels]]
for meta in sets:
    data_set[meta[0]] = {}
    data_set[meta[0]]['text'] = []
    data_set[meta[0]]['labels'] = []
    
    for ind, text in enumerate(meta[1]):
        data_set[meta[0]]['text'].append(text)
        data_set[meta[0]]['labels'].append(meta[2][ind])

        

        
data_set = DatasetDict({'train':Dataset.from_dict(data_set['train']),
                        'valid':Dataset.from_dict(data_set['val'])\
                       })

data_set

In [35]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    keys = ['attention_mask', 'input_ids']
    concatenated_examples = {k: sum(examples[k], []) for k in keys}
    total_length = len(concatenated_examples[list(keys)[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # this is needed as the used dataset is a subclass of ClassificationDataset, which requires label as a field...
    result["label"] = result["input_ids"].copy()
    result["labels"] = result["input_ids"].copy()
    return result

In [37]:
tokenized_datasets = data_set.map(tokenize_function, batched=True, batch_size=1000, num_proc=4, remove_columns=["text"])
tokenized_datasets_lm = tokenized_datasets.map(group_texts, batched=True, batch_size=1000, num_proc=4)
tokenized_datasets_lm = tokenized_datasets_lm.remove_columns(['label'])
#tokenized_datasets_lm['train'][0]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

{'labels': [2215,
  373,
  14821,
  2214,
  4583,
  4166,
  30,
  8241,
  373,
  262,
  717,
  20715,
  11596,
  8464,
  329,
  33818,
  30,
  2215,
  318,
  262,
  23637,
  605,
  2446,
  973,
  30,
  8241,
  15646,
  24300,
  377,
  30,
  2061,
  466,
  19062,
  8873,
  11799,
  4483,
  30,
  2437,
  1588,
  318,
  262,
  509,
  2224,
  8473,
  286,
  35618,
  4068,
  30,
  2215,
  373,
  31550,
  8757,
  7547,
  717,
  2716,
  30,
  2215,
  547,
  4171,
  4189,
  45938,
  3706,
  262,
  1181,
  15061,
  286,
  3936,
  30,
  8241,
  2727,
  262,
  2168,
  1012,
  1236,
  324,
  30,
  2215,
  373,
  262,
  23116,
  8121,
  3170,
  30,
  2061,
  338,
  262,
  3580,
  1022,
  582,
  12,
  6649,
  3637,
  290,
  19625,
  30,
  2215,
  750,
  6257,
  12770,
  717,
  10400,
  262,
  19752,
  286,
  262,
  11950,
  30,
  2061,
  318,
  262,
  4387,
  1582,
  10599,
  287,
  968,
  8936,
  30,
  2215,
  373,
  262,
  717,
  21401,
  338,
  24299,
  2716,
  30,
  2061,
  5873],
 'input_ids': 

In [46]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=2,
    #max_steps=300
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_lm['train'],
    eval_dataset= tokenized_datasets_lm['valid'],
)

In [42]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 68
  Batch size = 8


Perplexity: 58.93


In [48]:
trainer.train()

***** Running training *****
  Num examples = 516
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 130


Epoch,Training Loss,Validation Loss
1,No log,3.352275
2,No log,3.312449


***** Running Evaluation *****
  Num examples = 68
  Batch size = 8
***** Running Evaluation *****
  Num examples = 68
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=130, training_loss=3.557972599909856, metrics={'train_runtime': 1731.1987, 'train_samples_per_second': 0.596, 'train_steps_per_second': 0.075, 'total_flos': 33707280826368.0, 'train_loss': 3.557972599909856, 'epoch': 2.0})

In [49]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 68
  Batch size = 8


Perplexity: 27.45


In [50]:
model.save_pretrained("2-epoch_quest_english_finetune")


Configuration saved in 2-epoch_quest_english_finetune/config.json
Model weights saved in 2-epoch_quest_english_finetune/pytorch_model.bin


In [55]:
import torch
# Export contextual representations:

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")
device

def mean_pooling(model_output, attention_mask):
    # Mean Pooling - Take attention mask into account for correct averaging
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(model_output.size()).float()
    sum_embeddings = torch.sum(model_output * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

encoded = tokenizer(['Bromwell High is a cartoon comedy. It ran at the time as some other programs about school life, such as "Teachers".'],
                    return_tensors='pt')
encoded = {k:v.to(device) for k, v in encoded.items()}

model_output = model(**encoded, output_hidden_states=True, return_dict=True)
print(model_output.keys())
print(len(model_output['hidden_states'])) # contextual representations of separate words from each of the 6 layes
print(model_output['hidden_states'][-1].shape) # last layer with contextual representations (batch_size x num words x representation dim)

# Aggregate all the representations into one
mean_model_output = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])
mean_model_output.shape

odict_keys(['logits', 'past_key_values', 'hidden_states'])
7
torch.Size([1, 29, 768])


torch.Size([1, 768])

OLD STUFF

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
from datasets import load_metric
metric = load_metric('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    #learning_rate=2e-5,
    #weight_decay=0.01,
    num_train_epochs=2,
    do_train=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(200)),
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
)

In [15]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 50
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored